## <span style="color:#ff5f27">👩🏻‍🔬 Feature Engineering </span>

**Your Python Jupyter notebook should be configured for >8GB of memory.**

In this series of tutorials, we will build a recommender system for fashion items. It will consist of two models: a *retrieval model* and a *ranking model*. The idea is that the retrieval model should be able to quickly generate a small subset of candidate items from a large collection of items. This comes at the cost of granularity, which is why we also train a ranking model that can afford to use more features than the retrieval model.

### <span style="color:#ff5f27">✍🏻 Data</span>

We will use data from the [H&M Personalized Fashion Recommendations](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations) Kaggle competition.

<!-- https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/data

For this challenge you are given the purchase history of customers across time, along with supporting metadata. Your challenge is to predict what articles each customer will purchase in the 7-day period immediately after the training data ends. Customer who did not make any purchase during that time are excluded from the scoring. -->

The full dataset contains images of all products, but here we will simply use the tabular data. We have three data sources:
- `articles.csv`: info about fashion items.
- `customers.csv`: info about users.
- `transactions_train.csv`: info about transactions.


## <span style="color:#ff5f27">📝 Imports </span>

In [ ]:
# Hosted notebook environments may not have the local features package
import os

def need_download_modules():
    if 'google.colab' in str(get_ipython()):
        return True
    if 'HOPSWORKS_PROJECT_ID' in os.environ:
        return True
    return False

if need_download_modules():
    print("⚙️ Downloading modules...")
    os.system('mkdir -p features')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/recommender-system/features/articles.py')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/recommender-system/features/customers.py')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/recommender-system/features/transactions.py')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/recommender-system/features/ranking.py')        
    print('✅ Done!')
else:
    print("Local environment")

In [ ]:
try:
    from features.articles import prepare_articles
    from features.customers import prepare_customers
    from features.transactions import prepare_transactions
    from features.ranking import compute_ranking_dataset
except ImportError:
    print("⚙️ Downloading modules...")
    os.system('mkdir -p features')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/recommender-system/features/articles.py')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/recommender-system/features/customers.py')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/recommender-system/features/transactions.py')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/recommender-system/features/ranking.py')    
    print('✅ Done!')
    from features.articles import prepare_articles
    from features.customers import prepare_customers
    from features.transactions import prepare_transactions
    from features.ranking import compute_ranking_dataset    

In [ ]:
import pandas as pd
import numpy as np

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27">🗄️ Read Articles Data</span>

In [ ]:
articles_df = pd.read_parquet('https://repo.hops.works/dev/jdowling/articles.parquet')
print(articles_df.shape)
articles_df.head(3)

In [ ]:
# Check for NaNs
articles_df.isna().sum()[articles_df.isna().sum() > 0]

## <span style="color:#ff5f27">👨🏻‍🏭 Articles Feature Engineering</span>


In [ ]:
articles_df = prepare_articles(articles_df)
articles_df.head(3)

## <span style="color:#ff5f27">🗄️ Read Customers Data</span>

In [ ]:
customers_df = pd.read_parquet('https://repo.hops.works/dev/jdowling/customers.parquet')
print(customers_df.shape)
customers_df.head(3)

In [ ]:
# Check for NaNs
customers_df.isna().sum()[customers_df.isna().sum() > 0]

## <span style="color:#ff5f27">👨🏻‍🏭 Customers Feature Engineering</span>


In [ ]:
customers_df = prepare_customers(customers_df)
customers_df.head(3)

## <span style="color:#ff5f27">🗄️ Read Transactions Data</span>

In [ ]:
trans_df = pd.read_parquet('https://repo.hops.works/dev/jdowling/transactions_train.parquet')[:600000]
print(trans_df.shape)
trans_df.head(3)

In [ ]:
# Check for NaNs
trans_df.isna().sum()[trans_df.isna().sum() > 0]

## <span style="color:#ff5f27">👨🏻‍🏭 Transactions Feature Engineering</span>

The time of the year a purchase was made should be a strong predictor, as seasonality plays a big factor in fashion purchases. Here, we will use the month of the purchase as a feature. Since this is a cyclical feature (January is as close to December as it is to February), we'll map each month to the unit circle using sine and cosine.

In [ ]:
trans_df = prepare_transactions(trans_df)
trans_df.head(3)

In [ ]:
print(f"There are {len(trans_df):,} transactions in total.")

We can see that we have a large dataset. For the sake of the tutorial, we will use a small subset of this dataset, which we generate by sampling 25'000 customers and using their transactions.

In [ ]:
N_USERS = 25_000

# Consider only customers with age defined.
customers_df.dropna(inplace=True, subset=["age"])
customer_subset_df = customers_df.sample(N_USERS, random_state=27)

In [ ]:
trans_df = trans_df.merge(customer_subset_df["customer_id"])

print(f"Subset has {len(trans_df):,} transactions in total.")

## <span style="color:#ff5f27">🪄 Feature Group Creation </span>

A [feature group](https://docs.hopsworks.ai/feature-store-api/latest/generated/feature_group/) can be seen as a collection of conceptually related features.

Before we can create a feature group we need to connect to our feature store.

To create a feature group we need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group.

In [ ]:
customers_fg = fs.get_or_create_feature_group(
    name="customers",
    description="Customers data including age and postal code",
    version=1,
    primary_key=["customer_id"],
    online_enabled=True,
)

Here we have also set `online_enabled=True`, which enables low latency access to the data. A full list of arguments can be found in the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/feature_store_api/#create_feature_group).

At this point, we have only specified some metadata for the feature group. It does not store any data or even have a schema defined for the data. To make the feature group persistent we populate it with its associated data using the `save` function.

In [ ]:
customers_fg.insert(customers_df)

Let's do the same thing for the rest of the data frames.

In [ ]:
articles_fg = fs.get_or_create_feature_group(
    name="articles",
    description="Fashion items data including type of item, visual description and category",
    version=1,
    primary_key=["article_id"],
    online_enabled=True,
)
articles_fg.insert(articles_df)

In [ ]:
trans_fg = fs.get_or_create_feature_group(
    name="transactions",
    version=1,
    description="Transactions data including customer, item, price, sales channel and transaction date",
    primary_key=["customer_id", "article_id"], 
    online_enabled=True,
    event_time="t_dat",
)
trans_fg.insert(trans_df, write_options={"wait_for_job": True})

In [ ]:
ranking_df = compute_ranking_dataset(trans_fg, articles_fg, customers_fg)

In [ ]:
rank_fg = fs.get_or_create_feature_group(
    name="ranking",
    version=1,
    description="Derived feature group for ranking",
    primary_key=["customer_id", "article_id"], 
    parents=[articles_fg, customers_fg, trans_fg],
)
rank_fg.insert(ranking_df)

You should now be able to inspect the feature groups in the Hopsworks UI.

## <span style="color:#ff5f27">⚙️ Feature View Creation </span>

A [feature view](https://docs.hopsworks.ai/feature-store-api/latest/generated/feature_view/) can be seen as a logical view over a set of features that may come from different feature groups.

Feature views provides an Offline and Online API that can be used to generate training data or retrieve online feature vectors at inference time.

Now we can create two feature views for customers and articles, that will be used during model serving.

In [ ]:
customers_query = customers_fg.select_all()
fs.get_or_create_feature_view( 
    name='customers',
    query=customers_query,
    version=1,
)

In [ ]:
articles_query = articles_fg.select_all()
fs.get_or_create_feature_view(
    name='articles',
    query=articles_query,
    version=1,
)

In [ ]:
rank_fg = fs.get_or_create_feature_group(
    name="ranking",
    version=1)

In [ ]:
ranking_query = rank_fg.select_except(["customer_id", "article_id"])
fs.get_or_create_feature_view(
    name='ranking',
    query=ranking_query,
    labels = ["label"],
    version=1,
)

---
## <span style="color:#ff5f27">⏩️ Next Steps </span>
In the next notebook you'll train a retrieval model.